In [1]:
import torch
import torchtext
from torchvision.models.efficientnet import efficientnet_b1, EfficientNet_B1_Weights

In [16]:
model = efficientnet_b1(weights=None)
# model.classifier = torch.nn.Sequential(*list(model.classifier.children())[:-3])
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(0.2, True)
)

In [ ]:
class MultimodalModel(torch.nn.Module):
    def __init__(self, n_layers:int, embed_dim:int, hidden_dim:int, embedding:str="twitter.27B", bidirectionality:bool=False, freeze:bool=False, weights=None) -> None:
        super().__init__()
        model = efficientnet_b1(weights=weights)
        model.classifier = torch.nn.Sequential(torch.nn.Dropout(0.2,True))
        self.CNN = model

        glove_embeddings = torchtext.vocab.GloVe(embedding, embed_dim)
        self.LSTM = torch.nn.Sequential(
            torch.nn.Embedding.from_pretrained(glove_embeddings.vectors, freeze=freeze),
            torch.nn.LSTM(embed_dim, hidden_dim, n_layers, batch_first=True, bidirectional=bidirectionality),
        )

In [17]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat